In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, random_split

import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')

plt.rcParams["font.family"] = "D2Coding"
plt.rcParams["axes.unicode_minus"] = False

device = "cuda" if torch.cuda.is_available() else "cpu"



In [71]:
class TitanicDataset(Dataset):
    def __init__(self, dataframe, target_column=None, transform=None, is_train=True):
        self.dataframe = dataframe.copy()
        self.target_column = target_column
        self.transform = transform
        self.is_train = is_train
        
        self._preprocess()
        
        # / train_set과 test_set의 내용이 다름으로 블리안으로 처리 해야 함
        if self.is_train and target_column:
            self.targets = self.dataframe[target_column].values # sklearn과 다르게 값만 넣어야 함. 
            self.features = self.dataframe.drop([target_column], axis=1).values # sklearn과 다르게 값만 넣어야 함. 
        else:
            self.targets = None
            self.features = self.dataframe.values
        
    
    ########################### 반드시 숙지 할 것 #############################
    # 데이터 전처리 => 안하고 결과 보고 싶으면, 결정트리 앙상블로 돌림.
    # 외부에서 부르지 말라. 선언 _preprocess        
    def _preprocess(self):
        # 1. 불필요한 컬럼 삭제
        
        # 1-1. 삭제할 컬럼 선언
        columns_to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]
        # 1-2. 컬럼 있는지 확인
        existing_columns = [
            col for col in columns_to_drop if col in self.dataframe.columns
        ]
        
        # 1-3. 실질적으로 삭제
        if existing_columns:
            self.dataframe.drop(existing_columns, axis=1, inplace=True)

        # 2. 나이 결측값 처리 (중앙값으로 처리)
        if "Age" in self.dataframe.columns:
            self.dataframe["Age"].fillna(self.dataframe["Age"].median(), inplace=True)

        # 3. 승선항구 결측값 처리 (최빈값 - 최대로 많은 빈도(언급)가 나온 값)
        if "Embarked" in self.dataframe.columns:
            self.dataframe["Embarked"].fillna(
                self.dataframe["Embarked"].mode()[0], inplace=True
            )

        # 4. 요금 처리 (중앙값으로 처리)
        if "Fare" in self.dataframe.columns:
            self.dataframe["Fare"].fillna(self.dataframe["Fare"].median(), inplace=True)

        # 5. 새로운 특성 생성
        # 가족의 관계(부모님, 사촌 등의 관계 없는 애는 다 isAlone으로 만듦 => 이거 중요함.)
        if "SibSp" in self.dataframe.columns and "Parch" in self.dataframe.columns:
            self.dataframe["FamilySize"] = (
                self.dataframe["SibSp"] + self.dataframe["Parch"] + 1
            )
            self.dataframe["IsAlone"] = (self.dataframe["FamilySize"] == 1).astype(int)

        # 나이 그룹
        if "Age" in self.dataframe.columns:
            self.dataframe["AgeGroup"] = pd.cut(
                self.dataframe["Age"],
                bins=[0, 12, 18, 35, 60, 100],
                labels=[0, 1, 2, 3, 4],
            ).astype(int)
            
        # 요금 그룹
        if "Fare" in self.dataframe.columns:
            self.dataframe["FareGroup"] = pd.qcut(
                self.dataframe["Fare"], q=4, labels=[0, 1, 2, 3]
            ).astype(int)

        # 원-핫 인코딩
        if "Sex" in self.dataframe.columns:
            sex_dummies = pd.get_dummies(self.dataframe["Sex"], drop_first=True)
            self.dataframe = pd.concat([self.dataframe, sex_dummies], axis=1)
            self.dataframe.drop(["Sex"], axis=1, inplace=True)

        if "Embarked" in self.dataframe.columns:
            embarked_dummies = pd.get_dummies(
                self.dataframe["Embarked"], drop_first=True
            )
            self.dataframe = pd.concat([self.dataframe, embarked_dummies], axis=1)
            self.dataframe.drop(["Embarked"], axis=1, inplace=True)
        
        # 나머지 결측 (평균)
        self.dataframe.fillna(self.dataframe.mean(), inplace=True)
        print(f"전처리 후 특성 수: {len(self.dataframe.columns)}")
        print(f"특성 목록: {list(self.dataframe.columns)}")
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        features = self.features[idx]
        
        #변환 적용
        if self.transform:
            features = self.transform(features)
        
        features = torch.FloatTensor(features)
        
        if self.is_train and self.targets is not None:
            target = torch.LongTensor([self.targets[idx]])[0]
            return features, target
        else:
            return features

In [72]:
from sklearn.preprocessing import StandardScaler
class StandardScaleTransform:

    def __init__(self):
        self.scaler = StandardScaler()
        self.fitted = False

    def fit(self, data):
        self.scaler.fit(data)
        self.fitted = True
        return self

    def __call__(self, sample):
        if not self.fitted:
            raise ValueError(
                "스케일러가 아직 학습되지 않았습니다. fit() 메서드를 먼저 호출하세요."
            )

        if sample.ndim == 1:
            sample = sample.reshape(1, -1)
            return self.scaler.transform(sample).flatten()
        else:
            return self.scaler.transform(sample)

In [73]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

train_data = TitanicDataset(df_train, target_column="Survived")
test_data = TitanicDataset(df_test, is_train=False) # test_set에는 Survived 열이 없음 

transform = StandardScaleTransform()
transform.fit(train_data.features)

# 데이터를 숫자로 바꾸는 전처리
train_data.transform = transform
test_data.transform = transform

전처리 후 특성 수: 13
특성 목록: ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone', 'AgeGroup', 'FareGroup', 'male', 'Q', 'S']
전처리 후 특성 수: 12
특성 목록: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone', 'AgeGroup', 'FareGroup', 'male', 'Q', 'S']


In [74]:
train_dataset, val_dataset =  random_split(train_data, [0.2, 0.8])

In [75]:
batch_size = 32
tran_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [76]:
input_size = train_data.features.shape[1]
# 특성의 개수를 넣어야함. (사람 수가 아님)

In [ ]:
class TitanicNet(nn.Module):
    def __init__(self, input_size, hidden_size=[256, 128, 64], dropout_rate= 0.3): # 생성자의 매개변수가 가장 중요함. 
        super(TitanicNet, self).__init__()
        layers = []
        prev_size = input_size
        
        for i, hidden_size in enumerate(hidden_size):
            layers.extend([
                nn.Linear(input_size, 256), # 12개 집어 넣어서 256 나옴 => 이 숫자는 내가 정하는 것인데, cnn, lmst 같은 것들을 보면서 숫자는 익히는 것임.
                nn.BatchNorm1d(256), # 배치 정규화: 레이어로 들어가는 입력값이 한쪽으로 쏠리거나 너무 퍼지거나 너무 좁아지지 않게 해주는 것.
                nn.ReLU(), # 활성화 함수 렐루
                nn.Dropout(dropout_rate) # 50%의 확률로 아무거나 자름 # 학습 성능이 올라감 => 내려갈 수도 있음
            ])
            prev_size = hidden_size
        
        #출력층
        layers.append(nn.Linear(prev_size, 2))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x) # 네트워크라고 하는 순방향 모델 완성

In [78]:
input_size = train_data.features.shape[1]
model = TitanicNet(input_size)

In [ ]:
model # 입력측 + 은닉층까지가 1개층 / 총 은닉 3계층 만든 것임.

TitanicNet(
  (network): Sequential(
    (0): Linear(in_features=12, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=12, out_features=256, bias=True)
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=12, out_features=256, bias=True)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=64, out_features=2, bias=True)
  )
)